# Laboratorio 2 – Machine Learning  
## Complejidad y búsqueda de hiperparámetros – AlpesHearth  

### Integrantes:
1. Juan Sebastian Rodriguez Torres - 202214617 - js.rodriguezt1@uniandes.edu.co
2. Luis Felipe Sales Galviz - 202211531 - l.sales@uniandes.edu.co


## Carga de datos:

En esta sección vamos a cargar los datos para dejarlo con las modificaciones iniciales realizadas previas al modelado en el LAB 1:
Las columnas irrelevantes quedan eliminadas, los registros duplicados, las filas sin variable objetuvo y los "CVD Risk Score" menores a 0 y mayores a 100 tambien. Dado que el enfoque ahora está en modelado y validación, no se repetirá el proceso de exploración ni limpieza profunda que se hizo para cada modelo del lab 1.

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df_real = pd.read_csv("./data/Datos Lab 1.csv")
df = df_real.copy()

In [26]:
target = "CVD Risk Score"
cantNulos = df[target].isnull().sum()
df = df.dropna(subset=[target])
print(f"Registros con Risk Score nulo eliminados: {cantNulos}")

antes = df.shape[0]
df["Date of Service"] = pd.to_datetime(df["Date of Service"], errors="coerce")
df = df.sort_values(["Patient ID", "Date of Service"])
df = df.drop_duplicates(subset="Patient ID", keep="last")
desp = df.shape[0]
print(f"Duplicados eliminados: {antes - desp} (quedan {desp})")


df = df.drop(["Patient ID", "Date of Service", 
        "Blood Pressure (mmHg)", "Blood Pressure Category", 
        "CVD Risk Level"], axis =1)

df["CVD Risk Score"] = df["CVD Risk Score"].clip(0, 100)

df.head()

Registros con Risk Score nulo eliminados: 29
Duplicados eliminados: 261 (quedan 1349)


,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Total Cholesterol (mg/dL),HDL (mg/dL),Fasting Blood Sugar (mg/dL),Smoking Status,Diabetes Status,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Estimated LDL (mg/dL),CVD Risk Score
675,F,52.0,109.700,1.780,34.600,104.400,197.0,0.915,99.0,N,Y,High,N,178.000,0.587,103.0,99.0,NaN,18.010
1369,F,NaN,104.469,1.995,25.390,86.894,NaN,82.000,175.0,N,N,Low,Y,199.458,0.436,164.0,90.0,147.0,18.458
774,M,58.0,NaN,1.870,33.800,99.600,106.0,6.809,115.0,Y,Y,Low,N,187.000,0.533,NaN,91.0,33.0,16.530
1044,F,52.0,107.800,1.780,34.000,103.500,179.0,73.000,132.0,N,N,Low,Y,178.000,0.581,111.0,69.0,76.0,15.930
1290,M,57.0,116.772,1.569,34.126,90.737,254.0,60.000,177.0,N,N,Low,Y,156.854,0.578,117.0,116.0,164.0,17.755


## División del conjunto de datos

Para evaluar adecuadamente la capacidad de generalización de los modelos, el dataset será dividido en Conjunto de entrenamiento (75%) y Conjunto de prueba (25%)

El conjunto de entrenamiento se utilizará para el ajuste de modelos y la búsqueda de hiperparámetros.

El conjunto de prueba se utilizará al final para evaluar el desempeño del modelo que seleccionems.

## 1. Modelo de regresión polinomial
La regresión polinomial permite modelar relaciones no lineales mediante la expansión de las variables originales en términos de mayor grado.

1. Preprocesamiento
2. Generación de características polinomiales
3. Modelo de regresión lineal

Con GridSearchCV para explorar los diferentes grados del polinomio y distintas estrategias de escalamiento

El desempeño será evaluado mediante los mismos parametros: RMSE, MAE y R²

## 2. Curvas de validación
Para analizar el impacto de la complejidad, se generarán curvas de validación que muestren cómo varía el error en función del grado del polinomio.
Se grafica el rrror promedio en validación cruzada y la desviación estándar del error
El objetivo es identificarel puunto de mínima generalización, la evidencia de sobreajuste y la relación entre complejidad y estabilidad,


## 3. Modelos de regresión regularizados

La regularización son para controlar la magnitud de los coeficientes y reducir el sobreajuste.
Se usaran las tecnicas Ridge y Lasso

Se utilizará GridSearchCV para buscar diferentes valores del parámetro de penalización y estrategias de escalamiento

El análisis incluirá:
- Comparación de métricas
- Evaluación de estabilidad
- Análisis de magnitud de coeficientes
- Identificación de variables eliminadas por Lasso

## 4. Modelo polinomial regularizado

En esta sección se hara la generación de características polinomiales y la regularización.
Se explorarán simultáneamente los grados del polinomio, los parámetro de penalización y una estrategia para escalar.
El objetivo es evaluar si la regularización permite controlar el sobreajuste generado por el aumento en la complejidad.

## 5. Comparación y selección del mejor modelo

Haremos una tabla comparando: RMSE, MAE y R² y basados en los datos explicaremos cual es el mejor modelo.

## 6. Construcción de ntervalos de confianza


## 7. Análisis de resultados

### Análisis cuantitativo:

- ¿Cuál modelo obtuvo el mejor desempeño en test?
- ¿Coincide con el mejor promedio en validación cruzada?
- ¿El modelo con mejor promedio es necesariamente el más adecuado?
- ¿Cómo cambia el error con la complejidad?
- ¿Cómo afecta la regularización la estabilidad de los coeficientes?
- ¿Los intervalos de confianza sugieren estabilidad o alta variabilidad?

### Análisis cualitativo:

- Variables seleccionadas por Lasso.
- Interpretación práctica de los coeficientes.
- Diferencias entre precisión e interpretabilidad.
- Implicaciones estratégicas para AlpesHearth.
- Si mayor complejidad implica mayor valor organizacional.

## 8. Reflexión conceptual

## 9. Uso de IA generativa

Durante el desarrollo del laboratorio usamos modelos de IA generativa como herramienta de apoyo conceptual y para la estructuración del notebook.

Las decisiones de modificacion de datos y construccion de modelps, la interpretación de los resultados y los analisis fueron realizadas por el equipo, unicamente usando inteligencia artificial para confirmar conceptos o estructurar adecuadamente elementos repetitivos en los bloques de codigo para ahorrar tiempo.